In [1]:
import os, sys
# import numpy as np
from functools import partial
import autograd.numpy as np
from autograd import jacobian, grad, primitive
from scipy.stats import multivariate_normal

## <center>                                           Add module paths

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MomentMatching.StateModels import GaussianState
from MomentMatching.baseMomentMatch import UnscentedTransform, TaylorTransform, MonteCarloTransform
from MomentMatching.auto_grad import logpdf
# from MomentMatching.KalmanFilter import KalmanFilterSmoother

In [3]:
x = np.array([2.0, 0.1])

### <center> Define State Distribution

In [4]:
# xx_mean = np.array([1.0, 0.0], dtype=float)
# xx_sigma = np.array([[1, 0], [0, 1]], dtype=float)
xx_mean = 1e-5 * np.array([-0.7398, -0.6866], dtype=float)
xx_sigma = np.array([[0.0757,    0.0095],
                     [0.0095,    0.0701]], dtype=float)
state_distribution = GaussianState(xx_mean, xx_sigma)

xx_2_mean = 1e-5 *np.array([-0.3344, 0.1109], dtype=float)
xx_2_sigma =  np.array([[0.0757,    0.0095],
                        [0.0095,    0.0701]], dtype=float)
next_state_distribution = GaussianState(xx_2_mean, xx_2_sigma)

### <center>  Define a linear function 

In [5]:
A = np.array([[0.2, 0.8],
              [-0.8, 0.4]], dtype=float)

Q = 0.1 * np.eye(2)
R = Q
B = np.array([0.0, 0.0], dtype=float)
Z = A

A = np.array([[1, 0],
              [0, 0.1]], dtype=float)
B = np.array([0.0, 0.0], dtype=float)
Z = np.array([[2, 0],
              [0, 0.3]], dtype=float)

In [6]:
def measurement_function(x, A=A, B=B):
    return A @ x + B#[:, np.newaxis]

In [7]:
def transition_function(x, A=A, B=B):
    return measurement_function(x, A=A, B=B)
def identity_function(x, A=np.eye(2)):
    return A @ x

### <center> Define Taylor transform TT

In [8]:
TT = TaylorTransform(dimension_of_state=2)

In [9]:
TT.numerical_jacobian(f=measurement_function, x=xx_mean)

array([[ 0.2,  0.8],
       [-0.8,  0.4]])

### <center> 

``` python 
def measurement_update(self, node, measurement):

    assert isinstance(node, TimeSeriesNodeForEP)

    measurement_cavity_distribution = node.marginal / node.measurement_factor  # q(x_t) / q_up(x_t)

    new_node = node

    new_node.marginal = self.moment_matching(self.measurement, measurement_cavity_distribution, self.R, measurement)

    new_node.measurement_factor = new_node.marginal / measurement_cavity_distribution

    return new_node


```

In [10]:
meas_jac = jacobian(transition_function)
meas_jac(xx_mean)

array([[ 0.2,  0.8],
       [-0.8,  0.4]])

In [11]:
factor_mean = np.array([0.0, 0.0], dtype=float)
factor_sigma = 99999 * np.array([[1, 0], [0, 1]], dtype=float)
measurement_factor = GaussianState(factor_mean, factor_sigma)
measurement = np.array([2.0, 0.0], dtype=float)

In [12]:
fwd_factor_mean = 1.0e-05 * np.array([-0.6972, 0.3172], dtype=float)
fwd_factor_sigma = np.array([[0.1509,   0.0050],
                             [0.0050,   0.1536]], dtype=float)
fwd_factor = GaussianState(fwd_factor_mean, fwd_factor_sigma)

In [13]:
# logpdf = primitive(multivariate_normal.logpdf)

In [14]:
Q

array([[ 0.1,  0. ],
       [ 0. ,  0.1]])

In [15]:
def data_likelihood(f, mean, cov, measurement ):
    
    meanz = f(mean)   # z = f(x)
    linear_C = jacobian(f)(mean)  # linear factor A 
    covz = linear_C @ cov @ linear_C.T #  predictive covariance sz = var(f(x)) = A * Sigma * A^T
    logZi = logpdf(measurement, meanz, covz)
    return logZi

In [16]:
def data_likelihood3(f, mean1, cov1, mean2, cov2, Q= Q):
    
    meanx = f(mean1)   # z = f(x)
#     print(f'Mean is {meanx}')
    linear_C = jacobian(f)(mean1)  # linear factor A 
#     print(f'The value of jacobian in datalikelihood2 is {linear_C}')
    covx = linear_C @ cov1 @ linear_C.T + Q #  predictive covariance sz = var(f(x)) = A * Sigma * A^T
    print(f'The value of predicted cov in datalikelihood2 is {covx + cov2}')
    dlogZidSz = jacobian(logpdf, argnum=2)(mean2, meanx, covx + cov2)
    return logZi, dlogZidSz

In [17]:
def data_likelihood2(f, mean1, cov1, mean2, cov2, Q= Q):
    
    meanx = f(mean1)   # z = f(x)
#     print(f'Mean is {meanx}')
    linear_C = jacobian(f)(mean1)  # linear factor A 
#     print(f'The value of jacobian in datalikelihood2 is {linear_C}')
    covx = linear_C @ cov1 @ linear_C.T + Q #  predictive covariance sz = var(f(x)) = A * Sigma * A^T
#     print(f'The value of predicted cov in datalikelihood2 is {covx + cov2}')
    logZi = logpdf(mean2, meanx, covx + cov2)
    return logZi

In [18]:
# logZi_func = logpdf
# dlogZidMz_func = jacobian(logpdf, argnum=1)
# dlogZidSz_func = jacobian(logpdf, argnum=2)

# logZi = logZi_func(x=measurement, mean=mz, cov=sz)
# dlogZidMz = dlogZidMz_func(measurement, mz, sz)
# dlogZidSz = dlogZidSz_func(measurement, mz, sz) 

In [19]:
def moment_matching_gaussian(mxni, Sxni, dlogZidMz, dlogZidSz):
    mx = mxni + np.dot(Sxni, dlogZidMz.T)
    return mx

In [20]:
def ep_measurement_update( measurement_function, state_distribution,  z_measurement):
    measurement_cavity = state_distribution / measurement_factor
    logZi = data_likelihood(measurement_function, measurement_cavity.mean, measurement_cavity.cov, measurement)
    dlogZidMz = jacobian(data_likelihood, argnum=1)(measurement_function, measurement_cavity.mean, measurement_cavity.cov, z_measurement)
    dlogZidSz = jacobian(data_likelihood, argnum=2)(measurement_function, measurement_cavity.mean, measurement_cavity.cov, z_measurement)
    mx = measurement_cavity.mean + np.dot(measurement_cavity.cov, dlogZidMz.T)
#     print(dlogZidMz)
#     print(2 * dlogZidSz)
#     vx = measurement_cavity.cov + measurement_cavity.cov @ (2*dlogZidSz - np.dot(dlogZidMz.T, dlogZidMz) ) @ measurement_cavity.cov
    vx = measurement_cavity.cov - measurement_cavity.cov @  (  np.outer(dlogZidMz, dlogZidMz) - 2*dlogZidSz ) @ measurement_cavity.cov.T  
    filtered_distribution = GaussianState(mx, vx)
    return filtered_distribution
    

In [21]:
back_factor = measurement_factor
back_cavity = state_distribution / back_factor
fwd_cavity = next_state_distribution / fwd_factor
arg_tuple = (transition_function, back_cavity.mean, back_cavity.cov, fwd_cavity.mean, fwd_cavity.cov)
logZi = data_likelihood2 (*arg_tuple)  #okay
dlogZidMz = jacobian(data_likelihood2, argnum=1)(*arg_tuple)  # okay
dlogZidSz = jacobian(data_likelihood2, argnum=2)(*arg_tuple)
mx = back_cavity.mean + np.dot(back_cavity.cov, dlogZidMz.T)
vx = back_cavity.cov + back_cavity.cov @  ( - np.outer(dlogZidMz, dlogZidMz) + 2*dlogZidSz ) @ back_cavity.cov.T  
EP_smoother = GaussianState(mx, vx)

In [22]:
data_likelihood3 (*arg_tuple)

The value of predicted cov in datalikelihood2 is [[ 0.30520034  0.03597551]
 [ 0.03597551  0.28488647]]


(-0.60915277930685685, array([[-1.66302273,  0.21000679],
        [ 0.21000679, -1.78160479]]))

In [23]:
vx

array([[ 0.06141629,  0.00806166],
       [ 0.00806166,  0.05842577]])

In [24]:
dlogZidSz

array([[-1.27395015,  0.1864261 ],
       [ 0.1864261 , -1.21498697]])

In [25]:
np.outer(dlogZidMz,dlogZidMz)

array([[  2.65241775e-10,   2.26677197e-10],
       [  2.26677197e-10,   1.93719680e-10]])

In [26]:
def ep_back_update( transition_function, state_distribution,  next_state_distribution, factor ):
    back_cavity = state_distribution / factor
    fwd_cavity = next_state_distribution / factor
    arg_tuple = (transition_function, back_cavity.mean, back_cavity.cov, fwd_cavity.mean, fwd_cavity.cov)
    logZi = data_likelihood2 (*arg_tuple)
    dlogZidMz = jacobian(data_likelihood2, argnum=1)(*arg_tuple)
    dlogZidSz = jacobian(data_likelihood2, argnum=2)(*arg_tuple)
#     back_cavity = state_distribution
    mx = back_cavity.mean + np.dot(back_cavity.cov, dlogZidMz.T)
    print(dlogZidMz)
    print(back_cavity.cov @ (-2 * dlogZidSz + np.outer(dlogZidMz, dlogZidMz))  @ back_cavity.cov )
    print(back_cavity.cov)
#     vx = measurement_cavity.cov + measurement_cavity.cov @ (2*dlogZidSz - np.dot(dlogZidMz.T, dlogZidMz) ) @ measurement_cavity.cov
    vx = back_cavity.cov + back_cavity.cov @  ( - np.outer(dlogZidMz, dlogZidMz) + 2*dlogZidSz ) @ back_cavity.cov.T  
    filtered_distribution = GaussianState(mx, vx)
    return filtered_distribution

In [27]:
def measurement_update(self, measurement_function, state_distribution, z_measurement):

    z_mean, z_cov, xz_cross_cov = self.predict(measurement_function, state_distribution)
    # Add measurement Noise R_t
    z_cov = z_cov #+ self.system_model.R.cov

    K = np.matmul(xz_cross_cov, np.linalg.inv(z_cov))

    corrected_mean = state_distribution.mean + np.dot(K, (z_measurement - z_mean))  # equation 15  in Marc's ACC paper
    corrected_cov = state_distribution.cov - np.dot(K, np.transpose(xz_cross_cov))

    filtered_distribution = GaussianState(corrected_mean, corrected_cov)
    return filtered_distribution

In [28]:
def _smoother(self, transition_function, x_distribution, x_next_distribution, Q=Q):
    xx_mean, xx_cov, xx_cross_cov, = self.predict(transition_function, x_distribution)
    # Add transition Noise Q_t
    xx_cov = xx_cov + Q
    print(f'The predictions are mean: {xx_mean}, cov:{xx_cov}, cross_cov ={xx_cross_cov} ')
    # calculate smoother gain J_t
    J = np.matmul(xx_cross_cov, np.linalg.inv(xx_cov))
    print(f'The gain of smoother is {J}')
    smoothed_mean = x_distribution.mean + np.dot(J, (x_next_distribution.mean - xx_mean))
    smoothed_cov = x_distribution.cov + J @ (x_next_distribution.cov - xx_cov) @ J.T
    smoothed_distribution = GaussianState(smoothed_mean, smoothed_cov)
    return smoothed_distribution

In [29]:
corrected_ans = measurement_update(TT, measurement_function, state_distribution, measurement)

In [30]:
EP_ans = ep_measurement_update(measurement_function, state_distribution, measurement)

In [31]:
EP_ans == corrected_ans

True

In [32]:
EP_back = ep_back_update(transition_function, state_distribution, state_distribution, measurement_factor)

[  3.61518589e-05  -1.71782929e-05]
[[ 0.01749556  0.00144534]
 [ 0.00144534  0.01608979]]
[[ 0.07570006  0.00950001]
 [ 0.00950001  0.07010005]]


In [33]:
EP_back.mean

array([ -4.82450245e-06,  -7.72676155e-06])

In [34]:
smoother_ans = _smoother(TT, transition_function, state_distribution, next_state_distribution)

The predictions are mean: [ -6.97240000e-06   3.17200000e-06], cov:[[ 0.150932  0.005   ]
 [ 0.005     0.153584]], cross_cov =[[ 0.02274 -0.05676]
 [ 0.05798  0.02044]] 
The gain of smoother is [[ 0.16308268 -0.37487898]
 [ 0.38014765  0.12071089]]


In [35]:
smoother_ans.mean

array([ -6.03289547e-06,  -5.73569883e-06])

In [36]:
smoother_ans.mean

array([ -6.03289547e-06,  -5.73569883e-06])

In [37]:
xx_mean, xx_cov, xx_cross_cov, = TT.predict(transition_function, state_distribution)

In [38]:
xx_cov + Q

array([[ 0.150932,  0.005   ],
       [ 0.005   ,  0.153584]])

In [39]:
# J = state_distribution.cov @ Z @ np.linalg.pinv(x_cov)
# J @ (state_distribution.cov - x_cov) @ J.T
# state_distribution.mean + np.dot(J, (state_distribution.mean - xx_mean))

NameError: name 'x_cov' is not defined

In [ ]:
xx_mean

In [ ]:
jac(sz)

In [ ]:
test_cov = np.array(([[1.0, 0.5 ], [0.5, 1.0 ]]), dtype=float)
test_mean = np.array([0.0, 0.0], dtype=float)
data = test_mean + 0.1
A = np.random.randn(2,2)
A = A @ A.T

In [ ]:
def dummy_test(test_cov, A=A):    
    scaled_test_cov = A @ test_cov @ A.T
    return logpdf(data, test_mean, scaled_test_cov)

In [ ]:
jacobian(dummy_test)(test_cov)

In [42]:
EP_smoother.cov

array([[ 0.06141629,  0.00806166],
       [ 0.00806166,  0.05842577]])

In [40]:
EP_smoother == smoother_ans

True

In [41]:
smoother_ans.cov

array([[ 0.06141654,  0.00806106],
       [ 0.00806106,  0.05842459]])